In [16]:
from skilift import GTFSFeed, get_stop_node, read_osm, get_elevations_for_nodes, \
get_graph_nodes, ElevationAwareStreetDataset, OnEarthSurfaceNode, TransitEdgeProvider, StreetEdgeProvider

In [2]:
import pandas as pd
from zipfile import ZipFile
from collections import defaultdict
from typing import Dict, Set, Tuple, Iterable, Iterator, Any
import numpy as np
from shapely.geometry import Point, LineString, MultiLineString
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [3]:
fn = "/home/bmander/skilift_data/transit/redding.zip"

feed = GTFSFeed(fn)
transit_edge_provider = TransitEdgeProvider(feed)

In [4]:
# get epoch time on Aptil 19, 2023 at 1:00pm pacific time
t0 = pd.Timestamp("2023-04-19 13:00:00", tz="America/Los_Angeles")
t0

Timestamp('2023-04-19 13:00:00-0700', tz='America/Los_Angeles')

In [5]:
node0 = get_stop_node(feed, "Downtown Transit Center", t0)
node0

AtStopNode(stop_id:2000, datetime:2023-04-19 13:00:00-07:00)

In [8]:
node1 = transit_edge_provider.outgoing(node0)[0].node
node1

DepartureNode(pattern_id:5, service_id:c_1658_b_18260_d_31, row:1, col:0, datetime:2023-04-19 14:25:00-07:00)

In [9]:
node2 = transit_edge_provider.outgoing(node0)[0].node
node2

DepartureNode(pattern_id:5, service_id:c_1658_b_18260_d_31, row:1, col:0, datetime:2023-04-19 14:25:00-07:00)

In [10]:
transit_edge_provider.outgoing(node2)

[Edge(node=ArrivalNode(pattern_id:5, service_id:c_1658_b_18260_d_31, row:1, col:1, datetime:2023-04-19 14:35:00-07:00), weight=600.0)]

In [11]:
osm_filename = "/home/bmander/skilift_data/street/redding.pbf"


In [12]:
elev_geotiff_filename = "/home/bmander/skilift_data/elevation/USGS_13_n41w123_20210624.tif"

In [13]:
elev_dataset = ElevationAwareStreetDataset(osm_filename, elev_geotiff_filename)

Reading OSM file...done
Indexing ways...done
Getting node elevations...done
Creating spatial index...done


In [14]:
nd_points = elev_dataset.nodes.values()
lon = sum([x[0] for x in nd_points])/len(nd_points)
lat = sum([x[1] for x in nd_points])/len(nd_points)
lon, lat

(-122.35866061888524, 40.58422925237579)

In [15]:
node0 = OnEarthSurfaceNode(lon, lat, t0)
node0

OnEarthSurfaceNode(-122.35866061888524, 40.58422925237579, 2023-04-19 13:00:00-07:00)

In [17]:
street_edge_provider = StreetEdgeProvider(elev_dataset)
street_edge_provider.outgoing(node0)

[Edge(node=OnStreetNode(10557590, 13, 0.18715448062975482, 2023-04-19 13:00:11.591648613-07:00), weight=11.59164861368043)]